In [ ]:
import pandas as pd
import yfinance as yf
from datetime import date
import warnings
import os
from dotenv import load_dotenv

# Carregar as variáveis do arquivo .env
load_dotenv()

# Ignora avisos futuros para manter o output limpo
warnings.simplefilter(action='ignore', category=FutureWarning)

def ler_tickers_do_csv(caminho_do_arquivo: str, coluna_ticker: str = 'ticker') -> list:
    """Lê um arquivo CSV e extrai uma lista de tickers únicos."""
    try:
        df = pd.read_csv(caminho_do_arquivo)
        if coluna_ticker not in df.columns:
            print(f"Erro: A coluna '{coluna_ticker}' não foi encontrada no arquivo.")
            print(f"Colunas disponíveis: {list(df.columns)}")
            return []
        tickers = df[coluna_ticker].dropna().unique().tolist()
        return tickers
    except FileNotFoundError:
        print(f"Erro: O arquivo não foi encontrado em '{caminho_do_arquivo}'")
        return []
    except Exception as e:
        print(f"Ocorreu um erro ao ler o arquivo: {e}")
        return []

def gerar_tabela_comparativa_precos(lista_tickers: list, anos_anteriores: int = 7) -> tuple[pd.DataFrame, pd.DataFrame] | tuple[None, None]:
    """Busca o preço de fechamento ajustado para uma lista de tickers."""
    try:
        # Verifica se o ticker já tem .SA, se não tiver, adiciona
        tickers_sa = []
        for t in lista_tickers:
            ticker_upper = t.upper()
            if ticker_upper.endswith('.SA'):
                tickers_sa.append(ticker_upper)
            else:
                tickers_sa.append(f"{ticker_upper}.SA")
        
        hoje = date.today()
        ano_inicio = hoje.year - anos_anteriores
        
        print(f"Baixando dados históricos de {len(tickers_sa)} ativos...")
        hist = yf.download(tickers_sa, start=f"{ano_inicio}-01-01", end=hoje, auto_adjust=True, progress=False)
        
        if hist.empty:
            print("Nenhum dado histórico foi encontrado.")
            return None, None
            
        df_closes = hist['Close']
        
        # Lista para tabela completa (ticker, ano, fechamento)
        lista_completa = []
        # Lista para tabela resumida (ticker, fechamento_atual)
        lista_resumida = []
        
        for i, ticker in enumerate(lista_tickers):
            ticker_sa = tickers_sa[i]  # Usa o ticker já processado
            
            if ticker_sa not in df_closes.columns or df_closes[ticker_sa].dropna().empty:
                print(f"Aviso: Não foram encontrados dados para o ticker {ticker}. Pulando.")
                continue

            # Para tabela resumida - apenas fechamento atual
            fechamento_atual = df_closes[ticker_sa].iloc[-1].item()
            lista_resumida.append({
                'ticker': ticker,
                'fechamento_atual': fechamento_atual
            })
            
            # Para tabela completa - todos os anos
            # Adicionar fechamento atual
            lista_completa.append({
                'ticker': ticker,
                'ano': hoje.year,
                'fechamento': fechamento_atual
            })
            
            # Adicionar anos anteriores
            for j in range(anos_anteriores):
                ano_alvo = hoje.year - (j + 1)
                
                try:
                    dados_ano = df_closes.loc[str(ano_alvo)]
                    
                    if not dados_ano.empty and not dados_ano[ticker_sa].dropna().empty:
                        fechamento_ano = dados_ano[ticker_sa].iloc[-1].item()
                        lista_completa.append({
                            'ticker': ticker,
                            'ano': ano_alvo,
                            'fechamento': fechamento_ano
                        })
                except KeyError:
                    # Ano não encontrado nos dados - adiciona com None
                    lista_completa.append({
                        'ticker': ticker,
                        'ano': ano_alvo,
                        'fechamento': None
                    })

        if not lista_completa:
            print("Nenhum resultado foi processado com sucesso.")
            return None, None

        # Criar DataFrames
        df_completo = pd.DataFrame(lista_completa)
        df_resumido = pd.DataFrame(lista_resumida).set_index('ticker')
        
        return df_completo, df_resumido

    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None, None

# --- BLOCO DE EXECUÇÃO PRINCIPAL ---

# Pega o caminho do arquivo a partir da variável de ambiente
csv_path = os.getenv("FILTERED_COMPANIES_PATH")

if csv_path:
    print(f"Usando arquivo: {csv_path}")
    
    ativos_alvo = ler_tickers_do_csv(csv_path)

    if ativos_alvo:
        print(f"Processando {len(ativos_alvo)} ativos encontrados no arquivo: {', '.join(ativos_alvo[:5])}...")
        
        # Define o número de anos aqui para usar depois
        anos_para_analise = 7
        tabela_completa, tabela_resumida = gerar_tabela_comparativa_precos(ativos_alvo, anos_anteriores=anos_para_analise)

        if tabela_completa is not None and tabela_resumida is not None:
            print("\n--- Exibição Resumida (5 primeiras linhas) ---")
            
            # Exibir apenas 5 linhas da tabela resumida com 2 casas decimais
            tabela_exibicao = tabela_resumida.head(5).copy()
            tabela_exibicao['fechamento_atual'] = tabela_exibicao['fechamento_atual'].apply(
                lambda x: f"R$ {x:.2f}" if pd.notna(x) else "N/D"
            )
            print(tabela_exibicao.to_string())
            
            # Definir pasta de saída
            output_folder = r"E:\Github\Unicamp\Bussola-de-Valor\data"
            os.makedirs(output_folder, exist_ok=True)
            
            # Salvar tabela completa (ticker, ano, fechamento)
            output_path_completo = os.path.join(output_folder, "precos_acoes_completo.csv")
            tabela_completa_salvar = tabela_completa.copy()
            tabela_completa_salvar['fechamento'] = tabela_completa_salvar['fechamento'].round(2)
            tabela_completa_salvar.to_csv(output_path_completo, index=False)
            print(f"\n✅ Tabela completa salva em: {output_path_completo}")
            
            # Salvar tabela resumida (ticker, fechamento_atual)
            output_path_resumido = os.path.join(output_folder, "precos_acoes.csv")
            tabela_resumida_salvar = tabela_resumida.round(2)
            tabela_resumida_salvar.to_csv(output_path_resumido)
            print(f"✅ Tabela resumida salva em: {output_path_resumido}")
            
            # Estatísticas básicas
            print(f"\n📊 Estatísticas:")
            print(f"   • Total de ativos processados: {len(tabela_resumida)}")
            print(f"   • Período analisado: {anos_para_analise} anos")
            print(f"   • Total de registros na tabela completa: {len(tabela_completa)}")
            print(f"   • Data de execução: {date.today().strftime('%d/%m/%Y')}")
            
        else:
            print("Não foi possível gerar as tabelas de preços.")
    else:
        print("Nenhum ativo para processar. Verifique o conteúdo do arquivo CSV.")
else:
    print("ERRO: A variável de ambiente 'FILTERED_COMPANIES_PATH' não está definida no seu arquivo .env ou no sistema.")
    print("Certifique-se de que existe um arquivo .env com a linha:")
    print("FILTERED_COMPANIES_PATH=caminho/para/seu/arquivo.csv")
